In [94]:
import pandas as pd
import numpy as np
import shutil
import glob
import os
import datetime as dt
import pyswmm
import swmmio

# inputs

In [90]:
input_idf_table_file_path = './Assignment_3__Input_FE/Data_Centroid/BOM_IDF_Data/depths_-33.8774_151.093_all_design.csv'
input_temp_pattern_file_path = './Assignment_3__Input_FE/Data_Centroid/ECsouth/ECsouth_Increments.csv'
input_storm_stats_file_path = './Assignment_3__Input_FE/Data_Centroid/ECsouth/ECsouth_AllStats.csv'
output_dir = './Assignment_3__Output_FE__1st__centroid/'
original_inp_file_path = './Assignment_3__Input_FE/971007_SW5.INP'
simulation_starting_time = '2000/01/01 00:00:00'
stopping_time_after_precipitation_finish = {'days':0, 'hours':3, 'minutes':0, 'seconds':0}

simulation_starting_time = pd.to_datetime(simulation_starting_time)
stopping_time_after_precipitation_finish = pd.Timedelta(**stopping_time_after_precipitation_finish)

print(
    input_idf_table_file_path,
    input_temp_pattern_file_path,
    input_storm_stats_file_path,
    original_inp_file_path,
    simulation_starting_time,
    stopping_time_after_precipitation_finish,
    sep='\n'
    )

./Assignment_3__Input_FE/Data_Centroid/BOM_IDF_Data/depths_-33.8774_151.093_all_design.csv
./Assignment_3__Input_FE/Data_Centroid/ECsouth/ECsouth_Increments.csv
./Assignment_3__Input_FE/Data_Centroid/ECsouth/ECsouth_AllStats.csv
./Assignment_3__Input_FE/971007_SW5.INP
2000-01-01 00:00:00
0 days 03:00:00


In [91]:
def create_output_dir(arg_output_dir):
    """create output directory if it does not exist

    arguments:
        [string] --> arg_output_dir = path of the output directory name
    """
    if not os.path.exists(arg_output_dir):
        os.makedirs(arg_output_dir)

In [92]:
create_output_dir(output_dir)
output_dir

'./Assignment_3__Output_FE__1st__centroid/'

# setting up the inp file

In [93]:
inp_file_path = '{}inp_template.inp'.format(output_dir)
inp_file_path

'./Assignment_3__Output_FE__1st__centroid/inp_template.inp'

In [81]:
# copy the original inp file
shutil.copyfile(src=original_inp_file_path, dst=inp_file_path)

'./Assignment_3__Output_FE__1st__centroid/inp_template.inp'

In [ ]:
break

In [ ]:
with open(file=inp_file_path, mode='r+') as fp:
    # read an store all lines into list
    lines = fp.readlines()
    # move file pointer to the beginning of a file
    fp.seek(0)
    # truncate the file
    fp.truncate()

    # start writing lines
    # iterate line and line number
    for number, line in enumerate(lines):
        # delete line number 5 and 8
        # note: list index start from 0
        if number not in [4, 7]:
            fp.write(line)

# building idf table

In [5]:
idf_table = pd.read_csv(filepath_or_buffer=input_idf_table_file_path, skiprows=9)
frequency_label = idf_table.columns[2:].to_numpy()
idf_table.columns = idf_table.columns.to_series().apply(
    func=lambda arg: 
        '_'.join(arg.split(sep=' ')) 
        if arg[:8] == 'Duration' 
        else 'freq_' + '_'.join('_perc'.join('_'.join(arg.split(sep=' ')).split(sep='%')).split(sep='.'))
    )
frequency_tag = idf_table.columns[2:].to_numpy()
idf_table[['Duration', 'units']] = idf_table.apply(func=lambda arg: arg.Duration.split(sep=' '), axis=1, result_type='expand')
idf_table = idf_table[idf_table.columns[[0,-1] + list(range(1,len(idf_table.columns) - 1))]]
idf_table.Duration = idf_table.Duration.astype(float)
idf_table.to_csv(path_or_buf='{}table_idf_depths.csv'.format(output_dir), index=False)
idf_table.to_parquet(path='{}table_idf_depths.parquet'.format(output_dir))

idf_table

,Duration,units,Duration_in_min,freq_12EY,freq_6EY,freq_4EY,freq_3EY,freq_2EY,freq_63_2_perc,freq_50_perc,...,freq_20_perc,freq_0_2EY,freq_10_perc,freq_5_perc,freq_2_perc,freq_1_perc,freq_1_in_200,freq_1_in_500,freq_1_in_1000,freq_1_in_2000
0,1.0,min,1.0,0.999,1.13,1.37,1.55,1.80,2.28,2.51,...,3.25,3.31,3.74,4.22,4.84,5.32,5.82,6.58,7.17,7.76
1,2.0,min,2.0,1.770,2.00,2.39,2.67,3.07,3.77,4.10,...,5.14,5.25,5.86,6.57,7.52,8.25,9.08,10.30,11.30,12.20
2,3.0,min,3.0,2.410,2.73,3.29,3.68,4.25,5.23,5.71,...,7.21,7.35,8.23,9.24,10.60,11.60,12.80,14.50,15.80,17.10
3,4.0,min,4.0,2.960,3.36,4.07,4.58,5.30,6.56,7.19,...,9.15,9.33,10.50,11.80,13.50,14.90,16.30,18.40,20.10,21.80
4,5.0,min,5.0,3.430,3.91,4.76,5.37,6.23,7.75,8.52,...,10.90,11.10,12.60,14.10,16.20,17.80,19.50,22.10,24.00,26.10
5,10.0,min,10.0,5.150,5.91,7.28,8.27,9.70,12.20,13.60,...,17.60,18.00,20.40,23.00,26.40,29.00,31.60,35.80,39.00,42.20
6,15.0,min,15.0,6.290,7.25,8.96,10.20,12.00,15.30,16.90,...,22.00,22.50,25.50,28.70,33.00,36.20,39.60,44.70,48.70,52.80
7,20.0,min,20.0,7.160,8.26,10.20,11.70,13.70,17.50,19.40,...,25.20,25.70,29.10,32.80,37.70,41.40,45.20,51.20,55.70,60.40
8,25.0,min,25.0,7.860,9.07,11.20,12.80,15.10,19.30,21.40,...,27.70,28.30,31.90,36.00,41.30,45.30,49.60,56.10,61.20,66.30
9,30.0,min,30.0,8.460,9.76,12.10,13.80,16.30,20.80,23.00,...,29.70,30.30,34.20,38.60,44.30,48.60,53.20,60.20,65.60,71.10


# building temporal pattern table

In [6]:
temp_pattern_table = pd.read_csv(filepath_or_buffer=input_temp_pattern_file_path)
temp_pattern_table.rename(columns=lambda arg: arg.strip(), inplace=True)
# temp_pattern_table.rename(columns={'Duration':'Duration_in_min'}, inplace=True)
temp_pattern_table.columns = pd.Index(data=temp_pattern_table.columns[:5].to_list() + ['Increment_{:02}'.format(ind) for ind in range(len(temp_pattern_table.columns[5:]))])
temp_pattern_table.to_csv(path_or_buf='{}table_temp_patterns.csv'.format(output_dir), index=False)
temp_pattern_table.to_parquet(path='{}table_temp_patterns.parquet'.format(output_dir))

temp_pattern_table

,EventID,Duration,TimeStep,Region,AEP,Increment_00,Increment_01,Increment_02,Increment_03,Increment_04,...,Increment_46,Increment_47,Increment_48,Increment_49,Increment_50,Increment_51,Increment_52,Increment_53,Increment_54,Increment_55
0,4380,10,5,East Coast (South),frequent,58.06,41.94,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4382,10,5,East Coast (South),frequent,52.13,47.87,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4384,10,5,East Coast (South),frequent,60.71,39.29,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4385,10,5,East Coast (South),frequent,51.51,48.49,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4386,10,5,East Coast (South),frequent,54.55,45.45,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,2856,10080,180,East Coast (South),rare,1.14,1.80,1.36,1.29,3.20,...,0.01,0.01,0.02,0.03,0.32,0.17,0.52,0.00,0.02,0.00
716,5064,10080,180,East Coast (South),rare,0.61,0.93,0.26,0.04,2.91,...,5.27,0.31,0.02,1.21,1.57,0.08,0.05,0.15,0.40,0.11
717,5069,10080,180,East Coast (South),rare,0.00,0.00,0.00,1.06,0.04,...,0.22,0.20,0.57,0.06,2.06,0.00,0.03,0.18,0.12,3.09
718,5071,10080,180,East Coast (South),rare,1.21,0.00,0.00,0.00,0.00,...,0.05,0.56,0.70,3.31,0.10,0.11,0.00,0.00,0.00,0.15


In [7]:
temp_pattern_table.dtypes

EventID           int64
Duration          int64
TimeStep          int64
Region           object
AEP              object
                 ...   
Increment_51    float64
Increment_52    float64
Increment_53    float64
Increment_54    float64
Increment_55    float64
Length: 61, dtype: object

In [8]:
storm_stats_table = pd.read_csv(filepath_or_buffer=input_storm_stats_file_path)
storm_stats_table.rename(columns=lambda arg: arg.strip(), inplace=True)
for ind in [(' ','_'), ('(',''), (')',''), ('%','in_perc'), ('_No.','_Number'), ('_min','_in_min'), ('_mm','_in_mm')]:
    storm_stats_table.rename(columns=lambda arg: arg.replace(*ind), inplace=True)
# storm_stats_table.rename(columns={'Burst_Duration_in_min':'Duration_in_min'}, inplace=True)
storm_stats_table.dropna(axis=0, inplace=True)
storm_stats_table.Burst_Start_Date = pd.to_datetime(arg=storm_stats_table.Burst_Start_Date, format='%m/%d/%Y %H:%M')
storm_stats_table.Burst_End_Date = pd.to_datetime(arg=storm_stats_table.Burst_End_Date, format='%m/%d/%Y %H:%M')
storm_stats_table.Event_ID = storm_stats_table.Event_ID.astype(int)
storm_stats_table.Burst_Duration_in_min = storm_stats_table.Burst_Duration_in_min.astype(int)
storm_stats_table.Burst_Loading = storm_stats_table.Burst_Loading.astype(int)
storm_stats_table.DB_Event_Reference_Number = storm_stats_table.DB_Event_Reference_Number.astype(int)
storm_stats_table.DB_Pluviograph_Reference_Number = storm_stats_table.DB_Pluviograph_Reference_Number.astype(int)
storm_stats_table.to_csv(path_or_buf='{}table_storm_stats.csv'.format(output_dir), index=False)
storm_stats_table.to_parquet(path='{}table_storm_stats.parquet'.format(output_dir))

storm_stats_table

,Event_ID,Region,Region_source,Burst_Duration_in_min,Burst_Loading,Original_Burst_Depth_in_mm,AEP_Window,AEP_source_in_perc,Burst_Start_Date,Burst_End_Date,DB_Event_Reference_Number,DB_Pluviograph_Reference_Number,Offical_Gauge,Lat,Long
0,4380,East Coast (South),East Coast (South),10,1,15.50,frequent,14.4679,2009-03-25 17:45:00,2009-03-25 17:50:00,113650,1828,qcd_563064_233,-32.2375,150.6306
1,4382,East Coast (South),East Coast (South),10,1,18.80,frequent,14.4924,1999-12-28 17:05:00,1999-12-28 17:10:00,39777,517,qcd_061250,-32.6296,151.5919
2,4384,East Coast (South),East Coast (South),10,1,16.80,frequent,14.5144,2005-02-04 09:30:00,2005-02-04 09:35:00,42925,556,qcd_068102,-34.4869,150.4019
3,4385,East Coast (South),East Coast (South),10,2,18.50,frequent,15.6473,1999-03-27 22:30:00,1999-03-27 22:35:00,114063,1835,qcd_563079_231,-33.7944,150.5083
4,4386,East Coast (South),East Coast (South),10,2,11.00,frequent,47.6798,1993-04-28 14:05:00,1993-04-28 14:10:00,113366,1824,qcd_563056_231,-34.0333,150.2153
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,2856,East Coast (South),East Coast (North),10080,1,595.23,rare,1.0865,1996-05-01 00:05:00,1996-05-08 00:00:00,26738,351,qcd_040223,-27.4178,153.1142
716,5064,East Coast (South),East Coast (North),10080,2,589.88,rare,1.2899,1968-01-06 17:10:00,1968-01-13 17:05:00,24711,329,qcd_040093,-26.1831,152.6414
717,5069,East Coast (South),East Coast (North),10080,2,491.26,rare,2.7020,1968-01-06 04:10:00,1968-01-13 04:05:00,23354,313,qcd_039128,-24.9069,152.3230
718,5071,East Coast (South),East Coast (North),10080,2,400.84,rare,2.9538,1968-01-07 11:45:00,1968-01-14 11:40:00,27412,358,qcd_040318,-27.0258,152.5642


# building frequency classification table

In [9]:
ey_constants = np.array(object=[12,6,4,3,2,1,0.5,0.2])
ey_constants

array([12. ,  6. ,  4. ,  3. ,  2. ,  1. ,  0.5,  0.2])

In [10]:
aep_variables = np.multiply(np.add(1, np.negative(np.exp(np.negative(ey_constants)))),100)
aep_variables

array([99.99938558, 99.75212478, 98.16843611, 95.02129316, 86.46647168,
       63.21205588, 39.34693403, 18.12692469])

In [11]:
aep_constants = np.array(object=[50, 20, 10, 5, 2, 1, 0.5, 0.2, 0.1, 0.05, 0.02])
aep_constants

array([5.e+01, 2.e+01, 1.e+01, 5.e+00, 2.e+00, 1.e+00, 5.e-01, 2.e-01,
       1.e-01, 5.e-02, 2.e-02])

In [12]:
ey_variables = np.negative(np.log(np.add(1, np.negative(np.divide(aep_constants, 100)))))
ey_variables

array([6.93147181e-01, 2.23143551e-01, 1.05360516e-01, 5.12932944e-02,
       2.02027073e-02, 1.00503359e-02, 5.01254182e-03, 2.00200267e-03,
       1.00050033e-03, 5.00125042e-04, 2.00020003e-04])

In [13]:
ey = np.flip(m=np.sort(a=np.concatenate((ey_constants, ey_variables))))
ey

array([1.20000000e+01, 6.00000000e+00, 4.00000000e+00, 3.00000000e+00,
       2.00000000e+00, 1.00000000e+00, 6.93147181e-01, 5.00000000e-01,
       2.23143551e-01, 2.00000000e-01, 1.05360516e-01, 5.12932944e-02,
       2.02027073e-02, 1.00503359e-02, 5.01254182e-03, 2.00200267e-03,
       1.00050033e-03, 5.00125042e-04, 2.00020003e-04])

In [14]:
aep_percentage = np.flip(m=np.sort(a=np.concatenate((aep_constants, aep_variables))))
aep_percentage

array([9.99993856e+01, 9.97521248e+01, 9.81684361e+01, 9.50212932e+01,
       8.64664717e+01, 6.32120559e+01, 5.00000000e+01, 3.93469340e+01,
       2.00000000e+01, 1.81269247e+01, 1.00000000e+01, 5.00000000e+00,
       2.00000000e+00, 1.00000000e+00, 5.00000000e-01, 2.00000000e-01,
       1.00000000e-01, 5.00000000e-02, 2.00000000e-02])

In [15]:
aep_1_in_x = np.divide(100, aep_percentage)
aep_1_in_x

array([1.00000614e+00, 1.00248491e+00, 1.01865736e+00, 1.05239570e+00,
       1.15651764e+00, 1.58197671e+00, 2.00000000e+00, 2.54149408e+00,
       5.00000000e+00, 5.51665557e+00, 1.00000000e+01, 2.00000000e+01,
       5.00000000e+01, 1.00000000e+02, 2.00000000e+02, 5.00000000e+02,
       1.00000000e+03, 2.00000000e+03, 5.00000000e+03])

In [16]:
ari = np.divide(1,ey)
ari

array([8.33333333e-02, 1.66666667e-01, 2.50000000e-01, 3.33333333e-01,
       5.00000000e-01, 1.00000000e+00, 1.44269504e+00, 2.00000000e+00,
       4.48142012e+00, 5.00000000e+00, 9.49122158e+00, 1.94957257e+01,
       4.94983165e+01, 9.94991625e+01, 1.99499582e+02, 4.99499833e+02,
       9.99499917e+02, 1.99949996e+03, 4.99949998e+03])

In [17]:
aep_percentage

array([9.99993856e+01, 9.97521248e+01, 9.81684361e+01, 9.50212932e+01,
       8.64664717e+01, 6.32120559e+01, 5.00000000e+01, 3.93469340e+01,
       2.00000000e+01, 1.81269247e+01, 1.00000000e+01, 5.00000000e+00,
       2.00000000e+00, 1.00000000e+00, 5.00000000e-01, 2.00000000e-01,
       1.00000000e-01, 5.00000000e-02, 2.00000000e-02])

In [18]:
frequency_window = np.array(object=['frequent', 'intermediate', 'rare'])[
    np.add(np.digitize(x=aep_percentage, bins=np.array(object=[100, 14.4, 3.2, 0]), right=False), np.negative(1))
    ]
frequency_window

array(['frequent', 'frequent', 'frequent', 'frequent', 'frequent',
       'frequent', 'frequent', 'frequent', 'frequent', 'frequent',
       'intermediate', 'intermediate', 'rare', 'rare', 'rare', 'rare',
       'rare', 'rare', 'rare'], dtype='<U12')

In [19]:
frequency_tag = np.concatenate((frequency_tag, np.array(object=['freq_1_in_5000'])))
frequency_tag

array(['freq_12EY', 'freq_6EY', 'freq_4EY', 'freq_3EY', 'freq_2EY',
       'freq_63_2_perc', 'freq_50_perc', 'freq_0_5EY', 'freq_20_perc',
       'freq_0_2EY', 'freq_10_perc', 'freq_5_perc', 'freq_2_perc',
       'freq_1_perc', 'freq_1_in_200', 'freq_1_in_500', 'freq_1_in_1000',
       'freq_1_in_2000', 'freq_1_in_5000'], dtype=object)

In [20]:
frequency_label = np.concatenate((frequency_label, np.array(object=['1 in 5000'])))
frequency_label

array(['12EY', '6EY', '4EY', '3EY', '2EY', '63.2%', '50%', '0.5EY', '20%',
       '0.2EY', '10%', '5%', '2%', '1%', '1 in 200', '1 in 500',
       '1 in 1000', '1 in 2000', '1 in 5000'], dtype=object)

In [21]:
frequency_table = pd.DataFrame(
    data=np.column_stack(tup=(ey, aep_percentage, aep_1_in_x, ari, frequency_window, frequency_label, frequency_tag)),
    columns=['EY', 'AEP_percentage', 'AEP_1_in_x', 'ARI', 'freq_window', 'freq_label', 'freq_tag']
    )
frequency_table.to_csv(path_or_buf='{}table_frequency.csv'.format(output_dir), index=False)
frequency_table.to_parquet(path='{}table_frequency.parquet'.format(output_dir))
frequency_table

,EY,AEP_percentage,AEP_1_in_x,ARI,freq_window,freq_label,freq_tag
0,12.0,99.999386,1.000006,0.083333,frequent,12EY,freq_12EY
1,6.0,99.752125,1.002485,0.166667,frequent,6EY,freq_6EY
2,4.0,98.168436,1.018657,0.25,frequent,4EY,freq_4EY
3,3.0,95.021293,1.052396,0.333333,frequent,3EY,freq_3EY
4,2.0,86.466472,1.156518,0.5,frequent,2EY,freq_2EY
5,1.0,63.212056,1.581977,1.0,frequent,63.2%,freq_63_2_perc
6,0.693147,50.0,2.0,1.442695,frequent,50%,freq_50_perc
7,0.5,39.346934,2.541494,2.0,frequent,0.5EY,freq_0_5EY
8,0.223144,20.0,5.0,4.48142,frequent,20%,freq_20_perc
9,0.2,18.126925,5.516656,5.0,frequent,0.2EY,freq_0_2EY


In [22]:
r_ey = ey.copy()
r_ey[:14] = r_ey[:14].round(decimals=2)
r_ey[14:17] = r_ey[14:17].round(decimals=3)
r_ey[17:] = r_ey[17:].round(decimals=4)
r_ey

array([1.2e+01, 6.0e+00, 4.0e+00, 3.0e+00, 2.0e+00, 1.0e+00, 6.9e-01,
       5.0e-01, 2.2e-01, 2.0e-01, 1.1e-01, 5.0e-02, 2.0e-02, 1.0e-02,
       5.0e-03, 2.0e-03, 1.0e-03, 5.0e-04, 2.0e-04])

In [23]:
r_aep_percentage = aep_percentage.copy()
r_aep_percentage = r_aep_percentage.round(decimals=2)
r_aep_percentage

array([1.000e+02, 9.975e+01, 9.817e+01, 9.502e+01, 8.647e+01, 6.321e+01,
       5.000e+01, 3.935e+01, 2.000e+01, 1.813e+01, 1.000e+01, 5.000e+00,
       2.000e+00, 1.000e+00, 5.000e-01, 2.000e-01, 1.000e-01, 5.000e-02,
       2.000e-02])

In [24]:
r_aep_1_in_x = aep_1_in_x.copy()
r_aep_1_in_x[:2] = r_aep_1_in_x[:2].round(decimals=3)
r_aep_1_in_x[2:] = r_aep_1_in_x[2:].round(decimals=2)
r_aep_1_in_x

array([1.000e+00, 1.002e+00, 1.020e+00, 1.050e+00, 1.160e+00, 1.580e+00,
       2.000e+00, 2.540e+00, 5.000e+00, 5.520e+00, 1.000e+01, 2.000e+01,
       5.000e+01, 1.000e+02, 2.000e+02, 5.000e+02, 1.000e+03, 2.000e+03,
       5.000e+03])

In [25]:
r_ari = ari.copy()
r_ari = r_ari.round(decimals=2)
r_ari

array([8.0000e-02, 1.7000e-01, 2.5000e-01, 3.3000e-01, 5.0000e-01,
       1.0000e+00, 1.4400e+00, 2.0000e+00, 4.4800e+00, 5.0000e+00,
       9.4900e+00, 1.9500e+01, 4.9500e+01, 9.9500e+01, 1.9950e+02,
       4.9950e+02, 9.9950e+02, 1.9995e+03, 4.9995e+03])

In [26]:
r_frequency_table = pd.DataFrame(
    data=np.column_stack(tup=(r_ey, r_aep_percentage, r_aep_1_in_x, r_ari, frequency_window, frequency_label, frequency_tag)),
    columns=['EY', 'AEP_percentage', 'AEP_1_in_x', 'ARI', 'freq_window', 'freq_label', 'freq_tag']
    )
r_frequency_table.to_csv(path_or_buf='{}table_rounded_frequency.csv'.format(output_dir), index=False)
r_frequency_table.to_parquet(path='{}table_rounded_frequency.parquet'.format(output_dir))
r_frequency_table

,EY,AEP_percentage,AEP_1_in_x,ARI,freq_window,freq_label,freq_tag
0,12.0,100.0,1.0,0.08,frequent,12EY,freq_12EY
1,6.0,99.75,1.002,0.17,frequent,6EY,freq_6EY
2,4.0,98.17,1.02,0.25,frequent,4EY,freq_4EY
3,3.0,95.02,1.05,0.33,frequent,3EY,freq_3EY
4,2.0,86.47,1.16,0.5,frequent,2EY,freq_2EY
5,1.0,63.21,1.58,1.0,frequent,63.2%,freq_63_2_perc
6,0.69,50.0,2.0,1.44,frequent,50%,freq_50_perc
7,0.5,39.35,2.54,2.0,frequent,0.5EY,freq_0_5EY
8,0.22,20.0,5.0,4.48,frequent,20%,freq_20_perc
9,0.2,18.13,5.52,5.0,frequent,0.2EY,freq_0_2EY


# merge stats and pattern tables into timeseries

In [27]:
{key:val for key,val in enumerate(storm_stats_table.columns)}

{0: 'Event_ID',
 1: 'Region',
 2: 'Region_source',
 3: 'Burst_Duration_in_min',
 4: 'Burst_Loading',
 5: 'Original_Burst_Depth_in_mm',
 6: 'AEP_Window',
 7: 'AEP_source_in_perc',
 8: 'Burst_Start_Date',
 9: 'Burst_End_Date',
 10: 'DB_Event_Reference_Number',
 11: 'DB_Pluviograph_Reference_Number',
 12: 'Offical_Gauge',
 13: 'Lat',
 14: 'Long'}

In [28]:
df_storm_stats = storm_stats_table.copy()
df_storm_stats.drop(columns=storm_stats_table.columns[[1,2,4,5,7,8,9,10,11,12,13,14]], inplace=True)
df_storm_stats.rename(columns={'Event_ID':'event_id', 'Burst_Duration_in_min':'duration_in_min', 'AEP_Window':'freq_window'}, inplace=True)
df_storm_stats['id_duration'] = pd.Series(data=np.unique(ar=df_storm_stats.duration_in_min.to_numpy(), return_inverse=True)[1])
df_storm_stats['id_window'] = pd.Series(data=np.unique(ar=df_storm_stats.freq_window.to_numpy(), return_inverse=True)[1])
df_storm_stats['id_pattern'] = pd.Series(data=list(np.arange(stop=10))*int(len(df_storm_stats)/10))
df_storm_stats = df_storm_stats[df_storm_stats.columns[[0,4,3,5,2,1]]]

df_storm_stats

,event_id,id_window,id_duration,id_pattern,freq_window,duration_in_min
0,4380,0,0,0,frequent,10
1,4382,0,0,1,frequent,10
2,4384,0,0,2,frequent,10
3,4385,0,0,3,frequent,10
4,4386,0,0,4,frequent,10
...,...,...,...,...,...,...
715,2856,2,23,5,rare,10080
716,5064,2,23,6,rare,10080
717,5069,2,23,7,rare,10080
718,5071,2,23,8,rare,10080


In [29]:
df_temp_pattern = temp_pattern_table.copy()
# df_temp_pattern['time_series'] = df_temp_pattern.apply(func=lambda arg: arg.to_numpy()[5:], axis=1)
df_temp_pattern['time_series'] = df_temp_pattern.apply(func=lambda arg: [ind for ind in arg.to_list()[5:] if str(ind) != 'nan'], axis=1)
df_temp_pattern.drop(columns=df_temp_pattern.columns[5:-1], inplace=True)
df_temp_pattern.drop(columns='Region', inplace=True)
df_temp_pattern.rename(columns={'EventID':'event_id', 'Duration':'duration_in_min', 'TimeStep':'time_step', 'AEP':'freq_window'}, inplace=True)
df_temp_pattern['id_duration'] = pd.Series(data=np.unique(ar=df_temp_pattern.duration_in_min.to_numpy(), return_inverse=True)[1])
df_temp_pattern['id_timestep'] = pd.Series(data=np.unique(ar=df_temp_pattern.time_step.to_numpy(), return_inverse=True)[1])
df_temp_pattern['id_window'] = pd.Series(data=np.unique(ar=df_temp_pattern.freq_window.to_numpy(), return_inverse=True)[1])
df_temp_pattern['id_pattern'] = pd.Series(data=list(np.arange(stop=10))*int(len(df_temp_pattern)/10))
df_temp_pattern = df_temp_pattern[df_temp_pattern.columns[[0,7,5,6,8,3,1,2,4]]]

df_temp_pattern

,event_id,id_window,id_duration,id_timestep,id_pattern,freq_window,duration_in_min,time_step,time_series
0,4380,0,0,0,0,frequent,10,5,"[58.06, 41.94]"
1,4382,0,0,0,1,frequent,10,5,"[52.13, 47.87]"
2,4384,0,0,0,2,frequent,10,5,"[60.71, 39.29]"
3,4385,0,0,0,3,frequent,10,5,"[51.51, 48.49]"
4,4386,0,0,0,4,frequent,10,5,"[54.55, 45.45]"
...,...,...,...,...,...,...,...,...,...
715,2856,2,23,5,5,rare,10080,180,"[1.14, 1.8, 1.36, 1.29, 3.2, 4.78, 3.28, 3.13,..."
716,5064,2,23,5,6,rare,10080,180,"[0.61, 0.93, 0.26, 0.04, 2.91, 1.17, 4.05, 0.1..."
717,5069,2,23,5,7,rare,10080,180,"[0.0, 0.0, 0.0, 1.06, 0.04, 0.41, 0.71, 0.01, ..."
718,5071,2,23,5,8,rare,10080,180,"[1.21, 0.0, 0.0, 0.0, 0.0, 0.04, 0.52, 2.96, 1..."


In [30]:
{key:val for key,val in enumerate(df_temp_pattern)}

{0: 'event_id',
 1: 'id_window',
 2: 'id_duration',
 3: 'id_timestep',
 4: 'id_pattern',
 5: 'freq_window',
 6: 'duration_in_min',
 7: 'time_step',
 8: 'time_series'}

In [31]:
df_timeseries = pd.merge(left=df_storm_stats, right=df_temp_pattern, how='inner', on='event_id')# left_on='Event_ID', right_on='EventID')
df_timeseries.drop(columns=df_timeseries.columns[[0,6,7,9,10,11]], inplace=True)
df_timeseries.rename(columns=df_timeseries.columns[:5].to_series().str.rsplit(pat='_', n=1).str[0].to_dict(), inplace=True)
df_timeseries = df_timeseries[df_timeseries.columns[[0,1,5,2,3,4,6,7]]]

df_timeseries

,id_window,id_duration,id_timestep,id_pattern,freq_window,duration_in_min,time_step,time_series
0,0,0,0,0,frequent,10,5,"[58.06, 41.94]"
1,0,0,0,1,frequent,10,5,"[52.13, 47.87]"
2,0,0,0,2,frequent,10,5,"[60.71, 39.29]"
3,0,0,0,3,frequent,10,5,"[51.51, 48.49]"
4,0,0,0,4,frequent,10,5,"[54.55, 45.45]"
...,...,...,...,...,...,...,...,...
715,2,23,5,5,rare,10080,180,"[1.14, 1.8, 1.36, 1.29, 3.2, 4.78, 3.28, 3.13,..."
716,2,23,5,6,rare,10080,180,"[0.61, 0.93, 0.26, 0.04, 2.91, 1.17, 4.05, 0.1..."
717,2,23,5,7,rare,10080,180,"[0.0, 0.0, 0.0, 1.06, 0.04, 0.41, 0.71, 0.01, ..."
718,2,23,5,8,rare,10080,180,"[1.21, 0.0, 0.0, 0.0, 0.0, 0.04, 0.52, 2.96, 1..."


In [32]:
df_timeseries.duration_in_min.unique()#.size

array([   10,    15,    20,    25,    30,    45,    60,    90,   120,
         180,   270,   360,   540,   720,  1080,  1440,  1800,  2160,
        2880,  4320,  5760,  7200,  8640, 10080])

In [33]:
df_timeseries.id_duration.unique()#.size

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23])

In [34]:
{key:val for key,val in enumerate(df_timeseries)}

{0: 'id_window',
 1: 'id_duration',
 2: 'id_timestep',
 3: 'id_pattern',
 4: 'freq_window',
 5: 'duration_in_min',
 6: 'time_step',
 7: 'time_series'}

# idf depths

In [35]:
df_timeseries.columns

Index(['id_window', 'id_duration', 'id_timestep', 'id_pattern', 'freq_window',
       'duration_in_min', 'time_step', 'time_series'],
      dtype='object')

In [36]:
df_map = df_timeseries.copy()
df_map = df_map[['duration_in_min', 'time_step']]#.iloc[:,:2].drop_duplicates(subset=['time_step', 'duration_in_min'])
df_map.reset_index(drop=True, inplace=True)
df_map = {key:val for key,val in zip(df_map.duration_in_min, df_map.time_step)}

df_map

{10: 5,
 15: 5,
 20: 5,
 25: 5,
 30: 5,
 45: 5,
 60: 5,
 90: 5,
 120: 5,
 180: 15,
 270: 15,
 360: 15,
 540: 30,
 720: 30,
 1080: 60,
 1440: 60,
 1800: 120,
 2160: 120,
 2880: 120,
 4320: 180,
 5760: 180,
 7200: 180,
 8640: 180,
 10080: 180}

In [37]:
df_id = frequency_table.copy()
df_id.drop(index=len(df_id)-1, inplace=True)
df_id = df_id[['freq_window', 'freq_label', 'freq_tag']]#.iloc[:-1,4:]
df_id = pd.concat(objs=[
    pd.Series(data=np.full(shape=len(df_id), fill_value='depth')),
    df_id,
    pd.Series(data=np.unique(ar=df_id.freq_window.to_numpy(), return_inverse=True)[1], name='id_window'),
    df_id.index.to_series(name='id_tag')
    ], axis=1)
df_id = df_id[df_id.columns[[0,4,5,1,2,3]]]

df_id

,0,id_window,id_tag,freq_window,freq_label,freq_tag
0,depth,0,0,frequent,12EY,freq_12EY
1,depth,0,1,frequent,6EY,freq_6EY
2,depth,0,2,frequent,4EY,freq_4EY
3,depth,0,3,frequent,3EY,freq_3EY
4,depth,0,4,frequent,2EY,freq_2EY
5,depth,0,5,frequent,63.2%,freq_63_2_perc
6,depth,0,6,frequent,50%,freq_50_perc
7,depth,0,7,frequent,0.5EY,freq_0_5EY
8,depth,0,8,frequent,20%,freq_20_perc
9,depth,0,9,frequent,0.2EY,freq_0_2EY


In [38]:
df_idf = idf_table.copy()
df_idf = df_idf[df_idf.Duration_in_min >= 10]
df_idf.reset_index(drop=True, inplace=True)
df_idf.rename(columns={'Duration':'duration', 'Duration_in_min':'duration_in_min'}, inplace=True)
df_idf.duration_in_min = df_idf.duration_in_min.astype(int)
df_idf['id_duration'] = pd.Series(data=np.unique(ar=df_idf.duration_in_min.to_numpy(), return_inverse=True)[1])
df_idf['time_step'] = df_idf.duration_in_min.map(arg=df_map)
df_idf['id_timestep'] = pd.Series(data=np.unique(ar=df_idf.time_step.to_numpy(), return_inverse=True)[1])
df_idf = df_idf[df_idf.columns[-3:].to_list() + df_idf.columns[:-3].to_list()]
df_idf.set_index(keys=df_idf.columns[:6].to_list(), inplace=True)
df_idf.columns = pd.MultiIndex.from_frame(df=df_id)
df_idf = df_idf.stack(level=[1,2,3,4,5], future_stack=True)
df_idf.reset_index(inplace=True)
df_idf = df_idf[df_idf.columns[[6,0,2,7,3,4,8,5,1,9,10,11]]]

df_idf

,id_window,id_duration,id_timestep,id_tag,duration,units,freq_window,duration_in_min,time_step,freq_label,freq_tag,depth
0,0,0,0,0,10.0,min,frequent,10,5,12EY,freq_12EY,5.15
1,0,0,0,1,10.0,min,frequent,10,5,6EY,freq_6EY,5.91
2,0,0,0,2,10.0,min,frequent,10,5,4EY,freq_4EY,7.28
3,0,0,0,3,10.0,min,frequent,10,5,3EY,freq_3EY,8.27
4,0,0,0,4,10.0,min,frequent,10,5,2EY,freq_2EY,9.70
...,...,...,...,...,...,...,...,...,...,...,...,...
427,2,23,5,13,168.0,hour,rare,10080,180,1%,freq_1_perc,462.00
428,2,23,5,14,168.0,hour,rare,10080,180,1 in 200,freq_1_in_200,514.00
429,2,23,5,15,168.0,hour,rare,10080,180,1 in 500,freq_1_in_500,578.00
430,2,23,5,16,168.0,hour,rare,10080,180,1 in 1000,freq_1_in_1000,628.00


In [39]:
df_timeseries.columns

Index(['id_window', 'id_duration', 'id_timestep', 'id_pattern', 'freq_window',
       'duration_in_min', 'time_step', 'time_series'],
      dtype='object')

In [40]:
df_idf.columns

Index(['id_window', 'id_duration', 'id_timestep', 'id_tag', 'duration',
       'units', 'freq_window', 'duration_in_min', 'time_step', 'freq_label',
       'freq_tag', 'depth'],
      dtype='object', name=0)

# rainfall_patterns

In [59]:
rain_data = pd.merge(left=df_timeseries, right=df_idf, how='cross', suffixes=[None, '_'])
rain_data = rain_data[
    (rain_data.id_window == rain_data.id_window_) & 
    (rain_data.id_duration == rain_data.id_duration_) & 
    (rain_data.id_timestep == rain_data.id_timestep_)
    ]
rain_data.drop(columns=rain_data.columns[[8,9,10,14,15,16]], inplace=True)
rain_data = rain_data[rain_data.columns[[0,1,2,8,3,9,10,4,5,6,11,12,13,7]]]
rain_data.drop_duplicates(subset=rain_data.columns[:-1], inplace=True)
# rain_data.insert()
rain_data.sort_values(by=['id_tag', 'id_timestep', 'id_duration', 'id_pattern'], inplace=True)
# rain_data.sort_values(by=['id_pattern'], inplace=True)
rain_data.reset_index(drop=True, inplace=True)
rain_data['label'] = (
    'rain__' +
    (rain_data.index.astype(str).str.len().max() - rain_data.index.astype(str).str.len())*pd.Series(data=['0']*len(rain_data.index)) + rain_data.index.astype(str) + '__' +
    (rain_data.id_tag.astype(str).str.len().max() - rain_data.id_tag.astype(str).str.len())*pd.Series(data=['0']*len(rain_data.id_tag)) + rain_data.id_tag.astype(str) + '_' +
    (rain_data.id_timestep.astype(str).str.len().max() - rain_data.id_timestep.astype(str).str.len())*pd.Series(data=['0']*len(rain_data.id_timestep)) + rain_data.id_timestep.astype(str) + '_' +
    (rain_data.id_duration.astype(str).str.len().max() - rain_data.id_duration.astype(str).str.len())*pd.Series(data=['0']*len(rain_data.id_duration)) + rain_data.id_duration.astype(str) + '_' +
    (rain_data.id_pattern.astype(str).str.len().max() - rain_data.id_pattern.astype(str).str.len())*pd.Series(data=['0']*len(rain_data.id_pattern)) + rain_data.id_pattern.astype(str) + '_' +
    (rain_data.id_window.astype(str).str.len().max() - rain_data.id_window.astype(str).str.len())*pd.Series(data=['0']*len(rain_data.id_window)) + rain_data.id_window.astype(str) + '__' +
    (rain_data.freq_window.str.len().max() - rain_data.freq_window.str.len())*pd.Series(data=['_']*len(rain_data.freq_window)) + rain_data.freq_window + '__' +
    (rain_data.freq_tag.str.len().max() - rain_data.freq_tag.str.len())*pd.Series(data=['_']*len(rain_data.freq_tag)) + rain_data.freq_tag + '__' +
    (rain_data.time_step.astype(str).str.len().max() - rain_data.time_step.astype(str).str.len())*pd.Series(data=['0']*len(rain_data.time_step)) + rain_data.time_step.astype(str) + '__' +
    (rain_data.duration_in_min.astype(str).str.len().max() - rain_data.duration_in_min.astype(str).str.len())*pd.Series(data=['0']*len(rain_data.duration_in_min)) + rain_data.duration_in_min.astype(str) + '__' +
    (rain_data.id_pattern.astype(str).str.len().max() - rain_data.id_pattern.astype(str).str.len())*pd.Series(data=['0']*len(rain_data.id_pattern)) + rain_data.id_pattern.astype(str)
    )
rain_data = rain_data[rain_data.columns[:8].to_list() + rain_data.columns[-1:].to_list() + rain_data.columns[8:-1].to_list()]

rain_data

,id_window,id_duration,id_timestep,id_tag,id_pattern,duration,units,freq_window,label,duration_in_min,time_step,freq_label,freq_tag,depth,time_series
0,0,0,0,0,0,10.0,min,frequent,rain__0000__00_0_00_0_0______frequent_______fr...,10,5,12EY,freq_12EY,5.15,"[58.06, 41.94]"
1,0,0,0,0,1,10.0,min,frequent,rain__0001__00_0_00_1_0______frequent_______fr...,10,5,12EY,freq_12EY,5.15,"[52.13, 47.87]"
2,0,0,0,0,2,10.0,min,frequent,rain__0002__00_0_00_2_0______frequent_______fr...,10,5,12EY,freq_12EY,5.15,"[60.71, 39.29]"
3,0,0,0,0,3,10.0,min,frequent,rain__0003__00_0_00_3_0______frequent_______fr...,10,5,12EY,freq_12EY,5.15,"[51.51, 48.49]"
4,0,0,0,0,4,10.0,min,frequent,rain__0004__00_0_00_4_0______frequent_______fr...,10,5,12EY,freq_12EY,5.15,"[54.55, 45.45]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4315,2,23,5,17,5,168.0,hour,rare,rain__4315__17_5_23_5_2__________rare__freq_1_...,10080,180,1 in 2000,freq_1_in_2000,677.00,"[1.14, 1.8, 1.36, 1.29, 3.2, 4.78, 3.28, 3.13,..."
4316,2,23,5,17,6,168.0,hour,rare,rain__4316__17_5_23_6_2__________rare__freq_1_...,10080,180,1 in 2000,freq_1_in_2000,677.00,"[0.61, 0.93, 0.26, 0.04, 2.91, 1.17, 4.05, 0.1..."
4317,2,23,5,17,7,168.0,hour,rare,rain__4317__17_5_23_7_2__________rare__freq_1_...,10080,180,1 in 2000,freq_1_in_2000,677.00,"[0.0, 0.0, 0.0, 1.06, 0.04, 0.41, 0.71, 0.01, ..."
4318,2,23,5,17,8,168.0,hour,rare,rain__4318__17_5_23_8_2__________rare__freq_1_...,10080,180,1 in 2000,freq_1_in_2000,677.00,"[1.21, 0.0, 0.0, 0.0, 0.0, 0.04, 0.52, 2.96, 1..."


In [60]:
def build_df_timeseries(arg_time_series, arg_depth, arg_time_step, arg_sim_starting_time):

    rain_value = pd.Series(data=np.concatenate((
        np.array(object=[0]),
        np.divide(np.multiply(arg_depth, np.array(object=arg_time_series)), 100),
        np.array(object=[0])
        )), name='rain_value')

    rain_date = pd.Series(data=pd.date_range(
        start=arg_sim_starting_time, 
        periods=rain_value.size,
        freq=pd.Timedelta(minutes=arg_time_step)
        ), name='rain_date').dt.strftime(date_format='%m/%d/%Y')

    rain_time = pd.Series(data=pd.date_range(
        start=arg_sim_starting_time, 
        periods=rain_value.size,
        freq=pd.Timedelta(minutes=arg_time_step)
        ), name='rain_time').dt.strftime(date_format='%H:%M')

    return pd.concat(objs=[rain_date, rain_time, rain_value], axis=1)

In [61]:
rain_data.time_series = rain_data.apply(func=lambda arg: build_df_timeseries(arg.time_series, arg.depth, arg.time_step, simulation_starting_time), axis=1)
rain_data

,id_window,id_duration,id_timestep,id_tag,id_pattern,duration,units,freq_window,label,duration_in_min,time_step,freq_label,freq_tag,depth,time_series
0,0,0,0,0,0,10.0,min,frequent,rain__0000__00_0_00_0_0______frequent_______fr...,10,5,12EY,freq_12EY,5.15,rain_date rain_time rain_value 0 01/01/2...
1,0,0,0,0,1,10.0,min,frequent,rain__0001__00_0_00_1_0______frequent_______fr...,10,5,12EY,freq_12EY,5.15,rain_date rain_time rain_value 0 01/01/2...
2,0,0,0,0,2,10.0,min,frequent,rain__0002__00_0_00_2_0______frequent_______fr...,10,5,12EY,freq_12EY,5.15,rain_date rain_time rain_value 0 01/01/2...
3,0,0,0,0,3,10.0,min,frequent,rain__0003__00_0_00_3_0______frequent_______fr...,10,5,12EY,freq_12EY,5.15,rain_date rain_time rain_value 0 01/01/2...
4,0,0,0,0,4,10.0,min,frequent,rain__0004__00_0_00_4_0______frequent_______fr...,10,5,12EY,freq_12EY,5.15,rain_date rain_time rain_value 0 01/01/2...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4315,2,23,5,17,5,168.0,hour,rare,rain__4315__17_5_23_5_2__________rare__freq_1_...,10080,180,1 in 2000,freq_1_in_2000,677.00,rain_date rain_time rain_value 0 01/01...
4316,2,23,5,17,6,168.0,hour,rare,rain__4316__17_5_23_6_2__________rare__freq_1_...,10080,180,1 in 2000,freq_1_in_2000,677.00,rain_date rain_time rain_value 0 01/01...
4317,2,23,5,17,7,168.0,hour,rare,rain__4317__17_5_23_7_2__________rare__freq_1_...,10080,180,1 in 2000,freq_1_in_2000,677.00,rain_date rain_time rain_value 0 01/01...
4318,2,23,5,17,8,168.0,hour,rare,rain__4318__17_5_23_8_2__________rare__freq_1_...,10080,180,1 in 2000,freq_1_in_2000,677.00,rain_date rain_time rain_value 0 01/01...


In [ ]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

In [62]:
a = 0
b = a+60
rain_data.iloc[a:b]

,id_window,id_duration,id_timestep,id_tag,id_pattern,duration,units,freq_window,label,duration_in_min,time_step,freq_label,freq_tag,depth,time_series
0,0,0,0,0,0,10.0,min,frequent,rain__0000__00_0_00_0_0______frequent_______fr...,10,5,12EY,freq_12EY,5.15,rain_date rain_time rain_value 0 01/01/2...
1,0,0,0,0,1,10.0,min,frequent,rain__0001__00_0_00_1_0______frequent_______fr...,10,5,12EY,freq_12EY,5.15,rain_date rain_time rain_value 0 01/01/2...
2,0,0,0,0,2,10.0,min,frequent,rain__0002__00_0_00_2_0______frequent_______fr...,10,5,12EY,freq_12EY,5.15,rain_date rain_time rain_value 0 01/01/2...
3,0,0,0,0,3,10.0,min,frequent,rain__0003__00_0_00_3_0______frequent_______fr...,10,5,12EY,freq_12EY,5.15,rain_date rain_time rain_value 0 01/01/2...
4,0,0,0,0,4,10.0,min,frequent,rain__0004__00_0_00_4_0______frequent_______fr...,10,5,12EY,freq_12EY,5.15,rain_date rain_time rain_value 0 01/01/2...
5,0,0,0,0,5,10.0,min,frequent,rain__0005__00_0_00_5_0______frequent_______fr...,10,5,12EY,freq_12EY,5.15,rain_date rain_time rain_value 0 01/01/2...
6,0,0,0,0,6,10.0,min,frequent,rain__0006__00_0_00_6_0______frequent_______fr...,10,5,12EY,freq_12EY,5.15,rain_date rain_time rain_value 0 01/01/2...
7,0,0,0,0,7,10.0,min,frequent,rain__0007__00_0_00_7_0______frequent_______fr...,10,5,12EY,freq_12EY,5.15,rain_date rain_time rain_value 0 01/01/2...
8,0,0,0,0,8,10.0,min,frequent,rain__0008__00_0_00_8_0______frequent_______fr...,10,5,12EY,freq_12EY,5.15,rain_date rain_time rain_value 0 01/01/2...
9,0,0,0,0,9,10.0,min,frequent,rain__0009__00_0_00_9_0______frequent_______fr...,10,5,12EY,freq_12EY,5.15,rain_date rain_time rain_value 0 01/01/2...


In [77]:
indd = 100
indd = indd*18
rain_data.time_series[indd]

,rain_date,rain_time,rain_value
0,01/01/2000,00:00,0.00000
1,01/01/2000,00:30,23.82025
2,01/01/2000,01:00,1.82710
3,01/01/2000,01:30,1.16270
4,01/01/2000,02:00,20.03770
5,01/01/2000,02:30,1.37410
6,01/01/2000,03:00,0.86825
7,01/01/2000,03:30,0.29445
8,01/01/2000,04:00,1.32125
9,01/01/2000,04:30,0.29445


In [ ]:
indd = 100
indd

100

In [ ]:
rain_value = np.array(object=rain_pattern.time_series[indd])
rain_value

array([ 7.91,  4.67,  4.35,  2.32,  2.49,  5.51,  3.12,  3.12,  2.5 ,
        5.11,  2.11,  9.08,  5.56,  9.32,  8.12, 10.83,  8.42,  5.46])

In [ ]:
rain_depth = rain_pattern.depth[indd]
rain_depth

18.4

In [ ]:
rain_value = pd.Series(data=np.concatenate((
    np.array(object=[0]),
    np.divide(np.multiply(rain_depth, rain_value), 100),
    np.array(object=[0])
    )), name='rain_value')
rain_value

0     0.00000
1     1.45544
2     0.85928
3     0.80040
4     0.42688
5     0.45816
6     1.01384
7     0.57408
8     0.57408
9     0.46000
10    0.94024
11    0.38824
12    1.67072
13    1.02304
14    1.71488
15    1.49408
16    1.99272
17    1.54928
18    1.00464
19    0.00000
Name: rain_value, dtype: float64

In [ ]:
rain_value.size

20

In [ ]:
rain_value.sum()

18.4

In [ ]:
rain_duration = rain_pattern.duration_in_min[indd]
rain_duration

270

In [ ]:
rain_timestep = rain_pattern.time_step[indd]
rain_timestep

15

In [ ]:
rain_date = pd.Series(data=pd.date_range(
    start=simulation_starting_time, 
    periods=rain_value.size,
    freq=pd.Timedelta(minutes=rain_timestep)
    ), name='rain_date').dt.strftime(date_format='%m/%d/%Y')# %H:%M')
# date = date.dt.strftime()
rain_date

0     01/01/2000
1     01/01/2000
2     01/01/2000
3     01/01/2000
4     01/01/2000
5     01/01/2000
6     01/01/2000
7     01/01/2000
8     01/01/2000
9     01/01/2000
10    01/01/2000
11    01/01/2000
12    01/01/2000
13    01/01/2000
14    01/01/2000
15    01/01/2000
16    01/01/2000
17    01/01/2000
18    01/01/2000
19    01/01/2000
Name: rain_date, dtype: object

In [ ]:
rain_time = pd.Series(data=pd.date_range(
    start=simulation_starting_time, 
    periods=rain_value.size,
    freq=pd.Timedelta(minutes=rain_timestep)
    ), name='rain_time').dt.strftime(date_format='%H:%M')
# time = time.dt.strftime()
rain_time

0     00:00
1     00:15
2     00:30
3     00:45
4     01:00
5     01:15
6     01:30
7     01:45
8     02:00
9     02:15
10    02:30
11    02:45
12    03:00
13    03:15
14    03:30
15    03:45
16    04:00
17    04:15
18    04:30
19    04:45
Name: rain_time, dtype: object

In [ ]:
rain_timeseries = pd.concat(objs=[
    rain_date,
    rain_time,
    rain_value
    ], axis=1)
rain_timeseries

,rain_date,rain_time,rain_value
0,01/01/2000,00:00,0.00000
1,01/01/2000,00:15,1.45544
2,01/01/2000,00:30,0.85928
3,01/01/2000,00:45,0.80040
4,01/01/2000,01:00,0.42688
5,01/01/2000,01:15,0.45816
6,01/01/2000,01:30,1.01384
7,01/01/2000,01:45,0.57408
8,01/01/2000,02:00,0.57408
9,01/01/2000,02:15,0.46000


In [ ]:
orig_name = '{}asdasd.out'.format(output_dir)
file_name = '{}qweqweqwe.inp'.format(output_dir)

print(orig_name, file_name, sep='\n')
shutil.copyfile(src=orig_name, dst=file_name)

./Assignment_3__Output_FE__1st__centroid/asdasd.out
./Assignment_3__Output_FE__1st__centroid/qweqweqwe.inp


'./Assignment_3__Output_FE__1st__centroid/qweqweqwe.inp'

In [ ]:
for ind in rain_timeseries.index:
    with open(file=file_name, mode='r+') as file:
        writer = file.readlines()
        writer.insert(0+ind, '{}\t{}\t{}\n'.format(rain_timeseries.rain_date[ind], rain_timeseries.rain_time[ind], rain_timeseries.rain_value[ind]))
        file.seek(0)
        file.writelines(writer)

In [ ]:
date_time.dt.strftime(date_format='%m/%d/%Y %H:%M')

AttributeError: Can only use .dt accessor with datetimelike values

In [ ]:
dt.datetime?

Init signature: dt.datetime(self, /, *args, **kwargs)
Docstring:     
datetime(year, month, day[, hour[, minute[, second[, microsecond[,tzinfo]]]]])

The year, month and day arguments are required. tzinfo may be None, or an
instance of a tzinfo subclass. The remaining arguments may be ints.
File:           /usr/lib/python3.10/datetime.py
Type:           type
Subclasses:     ABCTimestamp, _NaT

In [ ]:
date_time.size

20

In [ ]:
pd.concat(objs=[
    pd.Series(data=date_time, name='date_time'),
    pd.Series(data=series, name='depth')
    ], axis=1)

,date_time,depth
0,2000-01-01 00:00:00,0.00000
1,2000-01-01 00:15:00,1.45544
2,2000-01-01 00:30:00,0.85928
3,2000-01-01 00:45:00,0.80040
4,2000-01-01 01:00:00,0.42688
5,2000-01-01 01:15:00,0.45816
6,2000-01-01 01:30:00,1.01384
7,2000-01-01 01:45:00,0.57408
8,2000-01-01 02:00:00,0.57408
9,2000-01-01 02:15:00,0.46000


In [ ]:
pd.date_range(
    start=simulation_starting_time, 
    end=simulation_starting_time + pd.Timedelta(minutes=duration) + pd.Timedelta(minutes=timestep),
    freq=pd.Timedelta(minutes=timestep)
    )

DatetimeIndex(['2000-01-01 00:00:00', '2000-01-01 00:15:00',
               '2000-01-01 00:30:00', '2000-01-01 00:45:00',
               '2000-01-01 01:00:00', '2000-01-01 01:15:00',
               '2000-01-01 01:30:00', '2000-01-01 01:45:00',
               '2000-01-01 02:00:00', '2000-01-01 02:15:00',
               '2000-01-01 02:30:00', '2000-01-01 02:45:00',
               '2000-01-01 03:00:00', '2000-01-01 03:15:00',
               '2000-01-01 03:30:00', '2000-01-01 03:45:00',
               '2000-01-01 04:00:00', '2000-01-01 04:15:00',
               '2000-01-01 04:30:00', '2000-01-01 04:45:00'],
              dtype='datetime64[ns]', freq='15T')

In [ ]:
type(np.array(object=[0]))

numpy.ndarray

In [ ]:
type(rain_pattern.time_series[0])

numpy.ndarray

In [ ]:
np.array(object=[np.nan, 1, 2, 3, np.nan])

array([nan,  1.,  2.,  3., nan])

In [ ]:
np.isnan(np.array(object=[np.nan, 1, 2, 3, np.nan]))

array([ True, False, False, False,  True])

In [ ]:
qwe = pd.Series(data=rain_pattern.time_series[0])
qwe

0     58.06
1     41.94
2       NaN
3       NaN
4       NaN
5       NaN
6       NaN
7       NaN
8       NaN
9       NaN
10      NaN
11      NaN
12      NaN
13      NaN
14      NaN
15      NaN
16      NaN
17      NaN
18      NaN
19      NaN
20      NaN
21      NaN
22      NaN
23      NaN
24      NaN
25      NaN
26      NaN
27      NaN
28      NaN
29      NaN
30      NaN
31      NaN
32      NaN
33      NaN
34      NaN
35      NaN
36      NaN
37      NaN
38      NaN
39      NaN
40      NaN
41      NaN
42      NaN
43      NaN
44      NaN
45      NaN
46      NaN
47      NaN
48      NaN
49      NaN
50      NaN
51      NaN
52      NaN
53      NaN
54      NaN
55      NaN
dtype: object

In [ ]:
qwe.dropna().to_numpy()

array([58.06, 41.94], dtype=object)

In [ ]:
qwe = rain_pattern.time_series[0].copy()
qwe

array([58.06, 41.94, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan], dtype=object)

In [ ]:
np.isnan(qwe)

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [ ]:
np.divide(np.multiply(rain_pattern.time_series[0], rain_pattern.depth[0]), 100)

array([2.9900900000000004, 2.15991, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan], dtype=object)

In [ ]:
np.concatenate((
    np.array(object=[0]),
    np.divide(np.multiply(rain_pattern.time_series[0], rain_pattern.depth[0]), 100),
    np.array(object=[0])
    ))

array([0, 2.9900900000000004, 2.15991, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, 0], dtype=object)

In [ ]:
rain_pattern.head()

,id_window,id_duration,id_timestep,id_tag,id_pattern,duration,units,freq_window,label,duration_in_min,time_step,freq_label,freq_tag,depth,time_series
0,0,0,0,0,0,10.0,min,frequent,rain__0000__00_0_00_0_0______frequent_______fr...,10,5,12EY,freq_12EY,5.15,"[58.06, 41.94, nan, nan, nan, nan, nan, nan, n..."
1,0,0,0,0,1,10.0,min,frequent,rain__0001__00_0_00_1_0______frequent_______fr...,10,5,12EY,freq_12EY,5.15,"[52.13, 47.87, nan, nan, nan, nan, nan, nan, n..."
2,0,0,0,0,2,10.0,min,frequent,rain__0002__00_0_00_2_0______frequent_______fr...,10,5,12EY,freq_12EY,5.15,"[60.71, 39.29, nan, nan, nan, nan, nan, nan, n..."
3,0,0,0,0,3,10.0,min,frequent,rain__0003__00_0_00_3_0______frequent_______fr...,10,5,12EY,freq_12EY,5.15,"[51.51, 48.49, nan, nan, nan, nan, nan, nan, n..."
4,0,0,0,0,4,10.0,min,frequent,rain__0004__00_0_00_4_0______frequent_______fr...,10,5,12EY,freq_12EY,5.15,"[54.55, 45.45, nan, nan, nan, nan, nan, nan, n..."


In [ ]:
rain_pattern

,id_window,id_duration,id_timestep,id_tag,id_pattern,duration,units,freq_window,label,duration_in_min,time_step,freq_label,freq_tag,depth,time_series
0,0,0,0,0,0,10.0,min,frequent,rain__0000__00_0_00_0_0______frequent_______fr...,10,5,12EY,freq_12EY,5.15,"[58.06, 41.94, nan, nan, nan, nan, nan, nan, n..."
1,0,0,0,0,1,10.0,min,frequent,rain__0001__00_0_00_1_0______frequent_______fr...,10,5,12EY,freq_12EY,5.15,"[52.13, 47.87, nan, nan, nan, nan, nan, nan, n..."
2,0,0,0,0,2,10.0,min,frequent,rain__0002__00_0_00_2_0______frequent_______fr...,10,5,12EY,freq_12EY,5.15,"[60.71, 39.29, nan, nan, nan, nan, nan, nan, n..."
3,0,0,0,0,3,10.0,min,frequent,rain__0003__00_0_00_3_0______frequent_______fr...,10,5,12EY,freq_12EY,5.15,"[51.51, 48.49, nan, nan, nan, nan, nan, nan, n..."
4,0,0,0,0,4,10.0,min,frequent,rain__0004__00_0_00_4_0______frequent_______fr...,10,5,12EY,freq_12EY,5.15,"[54.55, 45.45, nan, nan, nan, nan, nan, nan, n..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4315,2,23,5,17,5,168.0,hour,rare,rain__4315__17_5_23_5_2__________rare__freq_1_...,10080,180,1 in 2000,freq_1_in_2000,677.00,"[1.14, 1.8, 1.36, 1.29, 3.2, 4.78, 3.28, 3.13,..."
4316,2,23,5,17,6,168.0,hour,rare,rain__4316__17_5_23_6_2__________rare__freq_1_...,10080,180,1 in 2000,freq_1_in_2000,677.00,"[0.61, 0.93, 0.26, 0.04, 2.91, 1.17, 4.05, 0.1..."
4317,2,23,5,17,7,168.0,hour,rare,rain__4317__17_5_23_7_2__________rare__freq_1_...,10080,180,1 in 2000,freq_1_in_2000,677.00,"[0.0, 0.0, 0.0, 1.06, 0.04, 0.41, 0.71, 0.01, ..."
4318,2,23,5,17,8,168.0,hour,rare,rain__4318__17_5_23_8_2__________rare__freq_1_...,10080,180,1 in 2000,freq_1_in_2000,677.00,"[1.21, 0.0, 0.0, 0.0, 0.0, 0.04, 0.52, 2.96, 1..."


In [ ]:
a = 0
b = 60
rain_pattern.iloc[a:a+b]

,id_window,id_duration,id_timestep,id_tag,id_pattern,duration,units,freq_window,label,duration_in_min,time_step,freq_label,freq_tag,depth,time_series
0,0,0,0,0,0,10.0,min,frequent,rain__0000__00_0_00_0_0______frequent_______fr...,10,5,12EY,freq_12EY,5.15,"[58.06, 41.94, nan, nan, nan, nan, nan, nan, n..."
1,0,0,0,0,1,10.0,min,frequent,rain__0001__00_0_00_1_0______frequent_______fr...,10,5,12EY,freq_12EY,5.15,"[52.13, 47.87, nan, nan, nan, nan, nan, nan, n..."
2,0,0,0,0,2,10.0,min,frequent,rain__0002__00_0_00_2_0______frequent_______fr...,10,5,12EY,freq_12EY,5.15,"[60.71, 39.29, nan, nan, nan, nan, nan, nan, n..."
3,0,0,0,0,3,10.0,min,frequent,rain__0003__00_0_00_3_0______frequent_______fr...,10,5,12EY,freq_12EY,5.15,"[51.51, 48.49, nan, nan, nan, nan, nan, nan, n..."
4,0,0,0,0,4,10.0,min,frequent,rain__0004__00_0_00_4_0______frequent_______fr...,10,5,12EY,freq_12EY,5.15,"[54.55, 45.45, nan, nan, nan, nan, nan, nan, n..."
5,0,0,0,0,5,10.0,min,frequent,rain__0005__00_0_00_5_0______frequent_______fr...,10,5,12EY,freq_12EY,5.15,"[39.3, 60.7, nan, nan, nan, nan, nan, nan, nan..."
6,0,0,0,0,6,10.0,min,frequent,rain__0006__00_0_00_6_0______frequent_______fr...,10,5,12EY,freq_12EY,5.15,"[45.0, 55.0, nan, nan, nan, nan, nan, nan, nan..."
7,0,0,0,0,7,10.0,min,frequent,rain__0007__00_0_00_7_0______frequent_______fr...,10,5,12EY,freq_12EY,5.15,"[49.56, 50.44, nan, nan, nan, nan, nan, nan, n..."
8,0,0,0,0,8,10.0,min,frequent,rain__0008__00_0_00_8_0______frequent_______fr...,10,5,12EY,freq_12EY,5.15,"[30.38, 69.62, nan, nan, nan, nan, nan, nan, n..."
9,0,0,0,0,9,10.0,min,frequent,rain__0009__00_0_00_9_0______frequent_______fr...,10,5,12EY,freq_12EY,5.15,"[45.3, 54.7, nan, nan, nan, nan, nan, nan, nan..."


In [ ]:
{key:val for key, val in enumerate(rain_pattern.columns)}

{0: 'id_window',
 1: 'id_duration',
 2: 'id_timestep',
 3: 'id_tag',
 4: 'id_pattern',
 5: 'duration',
 6: 'units',
 7: 'freq_window',
 8: 'label',
 9: 'duration_in_min',
 10: 'time_step',
 11: 'freq_label',
 12: 'freq_tag',
 13: 'depth',
 14: 'time_series'}